In [2]:
import re
import string
import os
import pickle
import pickle5
import numpy as np
import copy
from collections import defaultdict
import pandas as pd
import sys
from nltk import pos_tag
import random

In [3]:
root = '/Users/christina/Documents/GitHub/parse-corpora/'
root = '/Users/gt/Documents/GitHub/parse-corpora/'
data = '/Volumes/GoogleDrive/My Drive/Research2020/McDermott/'
os.chdir(root)

In [8]:
sess = pd.read_csv(data+'sess_w_speech.csv')
aud = pd.read_csv(data+'aud_exps.csv')

In [9]:
aud

,Final Expt Name
0,AGL_Dehaene
1,NaturalSounds
2,Aph_lang_listen_v1
3,Aph_lang_listen_v2
4,Aph_nat_story
...,...
142,AliceLong_Vietnamese
143,AliceVidLoc
144,langloc_audio
145,langloc_video


In [10]:
sess

,UID,Session,Experiment
0,41,KAN_action_02,SN_ips232
1,41,KAN_action_02,DyLoc
2,41,KAN_action_02,Actions
3,41,KAN_action_02,ActionNaming
4,50,KAN_action_04,SN_ips232
...,...,...,...
707,828,FED_20200205c_3T2,ProdLoc_typing
708,831,FED_20200213b_3T2,LangLocAug2017_everySess
709,831,FED_20200213b_3T2,MDloc_ips224
710,831,FED_20200213b_3T2,ProdLoc


In [17]:
overlap = (sess.Experiment.isin(aud["Final Expt Name"]).astype(int))

In [18]:
overlap2 = (aud["Final Expt Name"].isin(sess.Experiment).astype(int))

In [20]:
np.sum(overlap)

62

In [21]:
np.sum(overlap2)

18

In [22]:
overlap.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
# get idx
g = np.arange(0,len(overlap))

In [32]:
i = g[overlap==1]

In [34]:
aud_sess = sess.iloc[i,:]

In [35]:
aud_sess

,UID,Session,Experiment
14,72,KAN_langsyntaxshapes_11 (KAN_syntshapes_11),SyntaxShapes
74,117,KAN_langsyntaxshapes_02,SyntaxShapes
80,118,KAN_langsyntaxshapes_03 (KAN_syntshapes_03),SyntaxShapes
98,122,KAN_langsyntaxshapes_05,SyntaxShapes
104,123,KAN_langsyntaxshapes_06 (KAN_syntshapes_06),SyntaxShapes
...,...,...,...
541,582,FED_20190726b_3T2,AliceVidLoc
565,687,FED_20190801a_3T2,AliceVidLoc
571,724,FED_20190802a_3T2,Music_2019
572,724,FED_20190802a_3T2,Music_viol_2019


In [43]:
np.unique(aud_sess.Experiment.values)

array(['AliceVidLoc', 'Articulation_Active', 'Articulation_Passive',
       'Articulation_kid', 'Hasson', 'ISCmega_Day', 'ISCmega_Dinner',
       'ISCmega_Elvis', 'ISCmega_Flute', 'ISCmega_Lisbon', 'ISCmega_Tree',
       'MNM', 'Music_2019', 'Music_viol_2019', 'PassAudio_ips232',
       'Stories_5_Elvis_T', 'SyntaxShapes', 'speechlangloc_Zeynep2014'],
      dtype=object)

In [44]:
len(np.unique(aud_sess.Experiment.values))

18